# <center>Python World Map Geovisualization Dashboard using Covid Data

### Import Libraries

In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [2]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

### Convert Data to JSON

In [3]:
covid1= json.loads(data)
covid1

{'Message': '',
 'Global': {'NewConfirmed': 237666,
  'TotalConfirmed': 33072057,
  'NewDeaths': 3591,
  'TotalDeaths': 997669,
  'NewRecovered': 210161,
  'TotalRecovered': 22921374},
 'Countries': [{'Country': 'Afghanistan',
   'CountryCode': 'AF',
   'Slug': 'afghanistan',
   'NewConfirmed': 35,
   'TotalConfirmed': 39227,
   'NewDeaths': 0,
   'TotalDeaths': 1453,
   'NewRecovered': 7,
   'TotalRecovered': 32642,
   'Date': '2020-09-29T01:54:18Z',
   'Premium': {}},
  {'Country': 'Albania',
   'CountryCode': 'AL',
   'Slug': 'albania',
   'NewConfirmed': 106,
   'TotalConfirmed': 13259,
   'NewDeaths': 2,
   'TotalDeaths': 377,
   'NewRecovered': 0,
   'TotalRecovered': 7397,
   'Date': '2020-09-29T01:54:18Z',
   'Premium': {}},
  {'Country': 'Algeria',
   'CountryCode': 'DZ',
   'Slug': 'algeria',
   'NewConfirmed': 153,
   'TotalConfirmed': 51067,
   'NewDeaths': 3,
   'TotalDeaths': 1714,
   'NewRecovered': 104,
   'TotalRecovered': 35860,
   'Date': '2020-09-29T01:54:18Z',
   '

### Normalize Data 

In [4]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,35,39227,0,1453,7,32642,2020-09-29T01:54:18Z
1,Albania,AL,albania,106,13259,2,377,0,7397,2020-09-29T01:54:18Z
2,Algeria,DZ,algeria,153,51067,3,1714,104,35860,2020-09-29T01:54:18Z
3,Andorra,AD,andorra,0,1836,0,53,0,1263,2020-09-29T01:54:18Z
4,Angola,AO,angola,46,4718,3,174,68,1707,2020-09-29T01:54:18Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,5,1074,0,35,0,999,2020-09-29T01:54:18Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-29T01:54:18Z
185,Yemen,YE,yemen,0,2030,0,587,6,1266,2020-09-29T01:54:18Z
186,Zambia,ZM,zambia,29,14641,0,332,57,13784,2020-09-29T01:54:18Z


### Convert to Pandas DataFrame

In [5]:
df = pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,35,39227,0,1453,7,32642,2020-09-29T01:54:18Z,{}
1,Albania,AL,albania,106,13259,2,377,0,7397,2020-09-29T01:54:18Z,{}
2,Algeria,DZ,algeria,153,51067,3,1714,104,35860,2020-09-29T01:54:18Z,{}
3,Andorra,AD,andorra,0,1836,0,53,0,1263,2020-09-29T01:54:18Z,{}
4,Angola,AO,angola,46,4718,3,174,68,1707,2020-09-29T01:54:18Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,5,1074,0,35,0,999,2020-09-29T01:54:18Z,{}
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-29T01:54:18Z,{}
185,Yemen,YE,yemen,0,2030,0,587,6,1266,2020-09-29T01:54:18Z,{}
186,Zambia,ZM,zambia,29,14641,0,332,57,13784,2020-09-29T01:54:18Z,{}


### Drop Unnecessary Columns

In [6]:
covid2 = df.drop(columns =['CountryCode','Slug','Date','Premium'],axis=1) 

### Generate Base Map

In [7]:
m = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
m

### Obtain Geodata

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

### Generate Choropleth Map Layer

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='COVID-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

### Generate Circular Markers

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [11]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [12]:
covid_final= pd.merge(covid2,coordinates,on='Country')

In [13]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)


In [14]:
covid_final.apply(plotDot, axis = 1)

m.fit_bounds(m.get_bounds())

m


### Generate Base Map

In [15]:
m1 = folium.Map(tiles='StamenToner', min_zoom=2)

m1

In [16]:
deaths=covid_final['TotalDeaths'].astype(float)

In [17]:
lat=covid_final['latitude'].astype(float)

In [18]:
lon=covid_final['longitude'].astype(float)

### Generate Heat Map Layer

In [19]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

### Generate Circular Marker

In [20]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)

covid_final.apply(plotDot, axis = 1)

m1.fit_bounds(m1.get_bounds())

m1